In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

**filtar dados: repetidos, muito semelhantes or inuteis**

por termos usado `&dedupValue=25&dedupField=url` e diferentes aliases, há informação repetida

problemas:

- ~~API has the following usage limits (250req/min, error 429):~~ time.sleep(60)

- ~~API error 404 for some urls:~~ return 0 (False) and skip it

- ~~extrair demora muito tempo, erro dá cabo de tudo:~~

- - funcao por coluna: inp: [col1, col2]: df03=02.copy, replace col, save, replace col, save...se der erro, só importa a ultima col

- - se fizer de vez keywords/sentiment/... assim poupa-se tempo do 1min dos requests

- noticias sobre algo, e contém a palavra Galp, como sei oq interessa?

- 1. se resumir a noticia e n tiver a palavra galp, ignoro?

- 2. selecionar frase que tem a palavra +-1 frase, e pronto ?

- 3. ?

- filtrar noticias muito parecidas

- 1. fazer NER, dois iguais, ver se é 80% igual?

- 2. selecionar 4 frases aleatórias ou contar palavras, e se for igual pronto?


tentar NER + Sentiment Analysis

In [10]:
def fetch_with_retry(linkToExtractedText):
    # Infinite loop to handle retry logic in case of 429 Too Many Requests
    while True:
        response = requests.get(linkToExtractedText)
        status_code = response.status_code
        
        if status_code == 200:
            # If the request is successful (200 OK), return the extracted text
            soup = BeautifulSoup(response.content, "html.parser")
            return soup.get_text()
        elif status_code == 429:
            # Handle 429 Too Many Requests by reading the Retry-After header
            print(f"Too many requests. Retrying after {60} seconds...")
            time.sleep(60)  # Pause execution for the retry period
        elif status_code == 404:
            return 0
        else:
            # For any other status codes (e.g., 500, ...), print the status and break the loop
            print(f"Request failed with status code {status_code}. Link was {linkToExtractedText}")
            break

In [6]:
"""
# Function to process each column
def process_cell(column, aliases):

    stats = {"404": 0, "duplicate": 0}

    filtered_column = []

    for row in aliases.index:
        filtered_cell = []
        seen_text = set()
        print(f"{row}", end = "; ")
        for i in column.loc[row]:
            
            # Extract text from 'linkToExtractedText'
            text = fetch_with_retry(i['linkToExtractedText'])
                

            # Skip if the text has already been processed
            if text in seen_text:
                stats["duplicate"] += 1
                continue

            elif not text: #ERROR 404
                stats["404"] += 1
                continue
            
            # Check if any alias is found in the text
            elif any(alias.lower() in text.lower() for alias in aliases.loc[row]):
                i["ExtractedText"] = text  # Add extracted text to the record
                
                # Remove unwanted fields
                i.pop('title', None)
                i.pop('snippet', None)
                i.pop('linkToExtractedText', None)
                
                # Append the processed record
                filtered_cell.append(i)
                
                # Mark this text as processed
                seen_text.add(text)

        filtered_column.append(filtered_cell)
                
    return filtered_column, stats


df = pd.read_parquet("data02.parquet")


start_time = datetime.now()
print(f"Starting: {start_time}")
for column in df.columns:
    if column == "aliases":
        pass
    else:
        print(f"\nProcessing {column}", end = ": ")
        df[column], stats = process_cell(df[column], df["aliases"])
end_time = datetime.now()
print(f"Ended: {end_time}.")
print(stats)

df.to_parquet("data03.parquet")
"""
""

''

In [5]:
df = pd.read_parquet("data01.parquet")

text =  fetch_with_retry(df["api.2000"].loc["Galp Energia"][1]["linkToExtractedText"])

import re

def extract_information(text, keyword):
    # Compile a regex pattern to find whole sentences containing the keyword
    pattern = re.compile(r'([^.]*?\b' + re.escape(keyword) + r'\b[^.]*\.)', re.IGNORECASE)
    
    # Find all matches for the keyword in the text
    matches = pattern.findall(text)
    
    # If matches are found, return them; otherwise, return an informative message
    if matches:
        return ' '.join(matches)
    else:
        return f"No information found about {keyword}."

# Example usage with your provided text

extract_information(text, "Galp")

' Alguns exemplos: Presidência do Conselho de Ministros, ministérios da Economia e das Finanças, Galp Energia, TAP, grupo EDP e RTP.  Refiram-se alguns exemplos sintomáticos: Presidência do Conselho de Ministros, ministérios da Economia e das Finanças, Galp Energia, TAP, grupo EDP e RTP.'

In [4]:
galp = pd.read_parquet("data01.parquet")["api.2000"].loc["Galp Energia"]

In [23]:
import spacy
from transformers import pipeline

# Carregar o modelo de NLP do spaCy
nlp = spacy.load("pt_core_news_sm")

# Inicializar o pipeline de sumarização
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="pt")


# Função para extrair partes do texto relacionadas à empresa e gerar um resumo
def extrair_resumo(texto, nome_empresa):
    # Passo 1: Identificar entidades mencionadas no texto
    doc = nlp(texto)
    # Passo 2: Filtrar sentenças que mencionam a empresa
    sentencas_relevantes = [sent.text for sent in doc.sents if nome_empresa.lower() in sent.text.lower()]
    # ISTO FAZ COM Q SE IGNORE TUDO  O Q N DIZ GALP !!!!
    
    # Passo 3: Se encontrarmos sentenças relacionadas à empresa, concatenamos e fazemos um resumo
    if sentencas_relevantes:
        texto_relevante = " ".join(sentencas_relevantes)
        # Gerar um resumo a partir do texto relevante
        resumo = summarizer(texto_relevante, max_length=len(texto_relevante)*0.5, min_length=30, do_sample=False) #max_length=100, min_length=30
        return resumo[0]['summary_text']
    
    return "Nenhuma informação relevante sobre a empresa foi encontrada."

# Exemplo de uso
#texto_exemplo = """
#A empresa X tem crescido no mercado de tecnologia. Ela recentemente fez uma parceria com a Y corp.
#No entanto, alguns críticos dizem que o mercado global pode impactar negativamente. Em 2023, a empresa X lançou novos produtos no setor de IA.
#"""

df = pd.read_parquet("data01.parquet")["api.2000"].loc["Galp Energia"]
texto_exemplo = fetch_with_retry(df[0]["linkToExtractedText"])


nome_empresa = "Galp"
resumo = extrair_resumo(texto_exemplo, nome_empresa)
print(resumo)


DN-Suplementos Negócios 10 de Setembro de 2001 Agências nacionais "esquecidas" pelo Estado Prevenção Rodoviária não convidou empresas portuguesas para o seu concurso publicitário Cátia Almeida DN-Natacha Cardoso Em pé de igualdade. Agências nacionais são tão boas como as multinacionais, afirma António Ferreira A Prevenção Rodoviária Portuguesa (PPR) abriu um concurso com o objectivo de seleccionar uma agência para desenvolver a sua próxima campanha de publicidade. Só que, para participar no concurso, apenas foram convidadas empresas que fazem parte de cadeias multinacionais (oito), ficando de lado as agências portuguesas. Para António Ferreira, director de serviço a clientes da MKT (agência do grupo Central de Comunicação), é inadmissível que, "num concurso com dinheiro dos contribuintes, as empresas nacionais não possam sequer apresentar uma proposta. Este caso, que não é único, ilustra uma atitude de preconceito para com aquilo que é nacional". "Há excelentes empresas portuguesas que

In [25]:
import spacy
from transformers import pipeline
import pandas as pd

# Carregar o modelo de NLP do spaCy
nlp = spacy.load("pt_core_news_sm")

# Inicializar o pipeline de sumarização
summarizer = pipeline("summarization")

# Função para extrair partes do texto relacionadas à empresa e gerar um resumo
def extrair_resumo(texto, nome_empresa):
    # Passo 1: Identificar entidades mencionadas no texto
    doc = nlp(texto)
    
    # Passo 2: Filtrar sentenças que mencionam a empresa
    sentencas_relevantes = [sent.text for sent in doc.sents if nome_empresa.lower() in sent.text.lower()]
    # MESMO PROBLEMA DE SO PEGAR EM FRASES Q DIZEM GALP !! 
    
    # Passo 3: Se encontrarmos sentenças relacionadas à empresa, concatenamos e fazemos um resumo
    if sentencas_relevantes:
        texto_relevante = " ".join(sentencas_relevantes)
        
        # Gerar um resumo a partir do texto relevante
        resumo = summarizer(texto_relevante, max_length=200, min_length=30, do_sample=False)
        
        return resumo[0]['summary_text']
    
    return "Nenhuma informação relevante sobre a empresa foi encontrada."

# Exemplo de uso com um DataFrame
df = pd.read_parquet("data01.parquet")["api.2000"].loc["Galp Energia"]

# Obter o texto de uma URL ou fonte
texto_exemplo = fetch_with_retry(df[0]["linkToExtractedText"])

# Extrair o resumo
nome_empresa = "Galp"
resumo = extrair_resumo(texto_exemplo, nome_empresa)
print(resumo)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 101. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


 Presidência do Conselho de Ministros, ministérios da Economia e das Finanças, Galp Energia, TAP, grupo EDP e RTP .
